In [42]:
!pip install kaggle

In [43]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [44]:
kaggle_dictionary=json.load(open("/content/kaggle (1).json"))

In [45]:
os.environ["KAGGLE_USERNAME"]=kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"]=kaggle_dictionary["key"]

In [46]:
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [47]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip  'kaggle (1).json'   sample_data


In [48]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip","r") as zip_ref:
  zip_ref.extractall()

In [49]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip  'kaggle (1).json'   sample_data


In [50]:
data=pd.read_csv("/content/IMDB Dataset.csv")

In [51]:
data.shape

(50000, 2)

In [52]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [53]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [54]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [55]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

<ipython-input-55-329ec005e21e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)


In [56]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [57]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [58]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)

In [59]:
print(train_data.shape,test_data.shape)

(40000, 2) (10000, 2)


In [60]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [61]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [62]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [63]:
Y_train=train_data["sentiment"]
Y_test=test_data["sentiment"]

In [64]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [65]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


In [66]:
model=Sequential()
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [67]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [68]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [69]:
model.fit(X_train,Y_train,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 137s 257ms/step - accuracy: 0.7177 - loss: 0.5383 - val_accuracy: 0.8541 - val_loss: 0.3486
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 133s 252ms/step - accuracy: 0.8203 - loss: 0.3994 - val_accuracy: 0.8134 - val_loss: 0.4066
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 254ms/step - accuracy: 0.8664 - loss: 0.3312 - val_accuracy: 0.8635 - val_loss: 0.3406
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 254ms/step - accuracy: 0.8895 - loss: 0.2816 - val_accuracy: 0.8644 - val_loss: 0.3379
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 126s 251ms/step - accuracy: 0.9040 - loss: 0.2491 - val_accuracy: 0.8612 - val_loss: 0.3379


In [71]:
loss,accuracy=model.evaluate(X_test,Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 90ms/step - accuracy: 0.8618 - loss: 0.3320
Test Loss: 0.3298325538635254
Test Accuracy: 0.8648999929428101


In [87]:
def predict_sentiment(review):
  sequence=tokenizer.texts_to_sequences([review])
  padded_sequence=pad_sequences(sequence,maxlen=200)
  prediction=model.predict(padded_sequence)
  sentiment="positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [91]:
new_review="This movie was fantastic. I loved it"
new_sentiment=predict_sentiment(new_review)
print(f"The sentiment of this review is: {new_sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
The sentiment of this review is: positive


In [90]:
new_review="This movie was ok but not that good."
new_sentiment=predict_sentiment(new_review)
print(f"The sentiment of this review is: {new_sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
The sentiment of this review is: negative


In [92]:
new_review="This movie was not that good."
new_sentiment=predict_sentiment(new_review)
print(f"The sentiment of this review is: {new_sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
The sentiment of this review is: negative


In [93]:
new_review="This movie was very good. I enjoyed a lot."
new_sentiment=predict_sentiment(new_review)
print(f"The sentiment of this review is: {new_sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
The sentiment of this review is: positive
